# Retrieval-Augmented Generation (RAG) Model for QA Bot on P&L Data (Part - 1)

## Importing the necessary libraries

In [1]:
import pdfplumber
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from pinecone import Pinecone, ServerlessSpec
import cohere

## Extracting Profit and Loss Data

In [2]:
# Load Embedding and Re-ranking Models
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Initialize Cohere API
cohere_api_key = "LC3f55pXtlMb5DAi4pW9PM57HxhfhMX0H89Sg6Xr"  # Replace with your Cohere API key
cohere_client = cohere.Client(cohere_api_key)

def extract_pnl_data(pdf_path):
    """Extracts P&L data from a PDF file (supports both tables and text parsing)."""
    data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Try extracting tables first
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if len(row) >= 2 and row[0] and row[-1]:
                        try:
                            description = row[0].strip()
                            value = float(row[-1].replace(',', '').replace('$', ''))
                            data.append((description, value))
                        except ValueError:
                            continue
            # If no tables found, fallback to text extraction
            text = page.extract_text()
            if text:
                for line in text.split("\n"):
                    parts = line.split()
                    if len(parts) >= 2:
                        try:
                            description = " ".join(parts[:-1])
                            value = float(parts[-1].replace(",", ""))
                            data.append((description, value))
                        except ValueError:
                            continue
    return pd.DataFrame(data, columns=["Description", "Value"])

# Load financial data
pdf_path = r"C:\Users\thegu\Downloads\Sample Financial Statement.pdf"  # Update with actual file path
financial_data_df = extract_pnl_data(pdf_path)

## Storing the embeddings

In [3]:
# Initialize Pinecone
pinecone_api_key = "pcsk_r5BQ5_8441hr1oD14nMzUKaLyhHxCLeotbVuNq1AwUhEWSGvkESctK3CrZ38LoEjW9CuZ"
pinecone_instance = Pinecone(api_key=pinecone_api_key)
index_name = "financial-qa"

# Create the index if it doesn't exist
if index_name not in pinecone_instance.list_indexes().names():
    pinecone_instance.create_index(
        name=index_name,
        dimension=384,  # all-MiniLM-L6-v2 generates 384-d embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pinecone_instance.Index(index_name)

# Generate embeddings and store in Pinecone
def store_embeddings(df):
    vectors = []
    for idx, row in df.iterrows():
        vector = embedding_model.encode(row["Description"]).tolist()
        metadata = {"description": row["Description"], "value": row["Value"]}
        vectors.append((f"entry_{idx}", vector, metadata))
    index.upsert(vectors)

store_embeddings(financial_data_df)

## Retrieving relevant parts of the data using ranking

In [4]:
def query_financial_data(query, top_k=5):
    query_vector = embedding_model.encode(query).tolist()
    results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    
    # Re-rank results
    descriptions = [match.metadata["description"] for match in results.matches]
    rerank_scores = reranker.predict([(query, desc) for desc in descriptions])
    ranked_results = sorted(zip(results.matches, rerank_scores), key=lambda x: x[1], reverse=True)
    
    context = "\n".join([f"{match.metadata['description']}: {match.metadata['value']}" for match, _ in ranked_results])
    return generate_response(query, context)

## Generate Natural Language Response with the help of Cohere-API

In [9]:
def generate_response(query, context):
    """Generates a response using Cohere with retrieved context."""
    prompt = f"""You are a financial analyst. Answer based on the given data.
    Provide all monetary values in Indian Rupees (₹).
    
    Context:
    {context}
    
    Question: {query}
    Answer (in ₹):"""
    
    # Send the prompt to Cohere's model for generation
    response = cohere_client.generate(
        model="command",  # You can specify other models as needed
        prompt=prompt,
        max_tokens=200,
        temperature=0.7
    )
    return response.generations[0].text.strip()

## Example Queries with Output

In [10]:
# Example Queries
queries = [
    "What is the gross profit for Q3 2024?",
    "How do the net income and operating expenses compare for Q1 2024?",
    "What is the total revenue?"
]

for query in queries:
    print(f"\nQuery: {query}")
    print("Response:", query_financial_data(query))


Query: What is the gross profit for Q3 2024?
Response: The gross profit from the provided data for Q3 2024 is 46,257₹.

Query: How do the net income and operating expenses compare for Q1 2024?
Response: Based on the provided information, the net income for the first quarter of 2024 appears to be higher than the operating expenses, however, the exact numerical values are not specified. 

The line-by-line breakdown of the net income is given as: 
1. Segment operating income: ₹8784.0
2. Other income, net: ₹4711.0
3. Other income, net: ₹2729.0

Whereas the operating expenses are divided into two parts with values given as: 
1. Total operating expenses: ₹13509.0
2. Income tax expense: ₹9214.0

To provide a exact answer, I would need the numerical values of the Q1 2024 operating expenses in order to do the comparison.

Query: What is the total revenue?
Response: The total revenue from the given information is Rs. 379016.0. 

To calculate this, you can sum the revenue from the software servi